# Neural decoding of eye movement trajectories

#### Summary:
The goal of **neural decoding** is to use patterns of brain activity to infer or "decode" information about the external world. By decoding neural activity to infer behavioral states, we can gain insights into how the brain processes information and generates behavior. This can help us understand the way in which information is represented and processed in the brain. The "behavior" we are interested in are eye movements! Eye movements make an excellent model system for understanding the neural mechanisms of motor control for two main reasons: 

1. The neural circuits involved in controlling eye movements are well-characterized and relatively simple compared to other motor systems. 
2. Eye movements are easy to measure and can be accurately recorded using invasive (eye coils) or non-invasive techniques (video-based eye tracking), making it possible to study the neural and behavioral aspects of motor control in a precise and detailed way.

#### Behavioral task:
In these experiments, a monkey was trained to follow a moving patch of dots on the screen. This propels them to make **smooth pursuit eye movements**, allowing them to smoothly track the motion of the stimulus on the screen and maintain visual fixation on it as it moves. Each time the patch of dots moves across the screen is called a "trial", and the monkey performs thousands of these trials in one recording session. And within one trial, which lasts ~2.851 seconds, we have neuronal and eye tracking data with a high temporal resolution of 1000 Hz (1 sample/ms). So when we stack together the recorded data from all of these trials, we end up with a *huge* dataset of 76,351 samples. 

#### Brain regions:
* **Medial temporal (MT) area:** brain region that is involved in processing visual motion information, containing neurons that are particularly sensitive to the motion (speed and direction) of moving objects in the visual field.
* **Frontal eye fields (FEF):** brain region that is involved in the selection of visual targets and the planning and execution of eye movements towards those targets, containing neurons that are sensitive to a variety of visual and non-visual stimuli relevant to the control of eye movements.

#### Objective:
The central goal of this project is to use neural activity from regions of the brain that are involved in controlling eye movements to reconstruct eye trajectories. For this specific part of the project, we want to understand how much information populations of neurons from 2 brain regions (FEF and MT) contain regarding the position, velocity, and acceleration of the eyes. 
Some stepping-stone questions include:
1. How much information do FEF neurons contain regarding the position, velocity, and acceleration of the eyes?
2. How much information do MT neurons contain regarding the position, velocity, and acceleration of the eyes?
3. How much information do combinations of FEF and MT neurons contain regarding the position, velocity, and acceleration of the eyes?

If FEF and MT neurons contain redundant information, then we would expect all 3 of the questions to result in the same decoding performance. But perhaps, FEF neurons would care more about the position of the eyes relative to the visual target and MT neurons would be more sensitive to the velocity of the eyes? That's what we want to find out. 

#### Data
There are two pickle files included in this notebook, both from a single recording session: one contains the neural data and eye traces and the other with specific details about the neurons. 
The data pickle file consists of four matrices:
1. Neural activity (N samples x D neurons)
2. Horizontal and vertical eye positions (N samples x 2)
3. Horizontal and vertical eye velocities (N samples x 2)
4. Horizontal and vertical eye accelerations (N samples x 2). 

The unit properties data file consists of one matrix, where each row is a single neuron (with the same indices as the D neurons in the other pickle file):
* UnitName = name of this unit, in alphabetical order (same order as neural_data)
* BrainArea = either FEF or MT
* SNR = signal-to-noise ratio, measure of the strength of a neuron's electrical signal relative to the background noise in the recording (higher SNR = clearer signal that is easier to detect and analyze)
* BestDir = neurons' preferred direction, or the direction of motion it fires (on-average) the most
* MeanFR_BestDir = the trial-averaged mean firing rate of that neuron in its' preferred direction, in Hz
* VarFR_BestDir = the trial-averaged variance in firing rate of that neuron in its' preferred direction

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
get_ipython().run_line_magic('matplotlib', 'inline')
import sys
import os
np.set_printoptions(threshold=sys.maxsize)

cwd = os.getcwd()
sys.path.append(os.path.join(cwd, '..', 'handy_functions'))

from scipy import io
from scipy import stats
import pickle
import time
import pandas as pd

from preprocessing_funcs import get_spikes_with_history
from metrics import get_R2
from metrics import get_rho
from decoders import WienerCascadeDecoder
from decoders import WienerFilterDecoder
from decoders import DenseNNDecoder
from decoders import SimpleRNNDecoder
from decoders import GRUDecoder
from decoders import LSTMDecoder
from decoders import XGBoostDecoder
from decoders import SVRDecoder


from sklearn import linear_model 
from sklearn.svm import SVR 
from sklearn.svm import SVC 
from bayes_opt import BayesianOptimization

# Pickle file #1
with open(os.path.join(cwd, '..', 'datasets/')+'vars-pa29dir4A-pre500-post300-dt50.pickle','rb') as f:
    neural_data,eye_pos,eye_vel,eye_acc=pickle.load(f,encoding='latin1') 

print('Number of samples = {}'.format(neural_data.shape[0]))
print('Number of neurons = {}'.format(neural_data.shape[1]))
   
# Pickle file #2
units = pd.read_csv(os.path.join(cwd,'..','datasets/')+'units-pa29dir4A-pre500-post300.csv')
units.head(10)   

In [ ]:
# Function for plotting true and predicted eye traces
def plot_eyeTraces(first_sample,last_sample,y_true,y_predicted,output_type):
    ts=np.arange(int(first_sample),int(last_sample))
    fig, ax = plt.subplots(2,1,figsize=(10,10))

    x = (ts*50)/1000
    m = 0
    ax[m].plot(x,y_predicted[ts,0],'b',label='predicted') 
    ax[m].plot(x,y_true[ts,0],'k',label='true') 
    ax[m].tick_params(direction='in') 
    ax[m].set_xticklabels('')
    ax[m].legend()
    ax[m].set(ylabel='horizontal eye {}'.format(output_type))
    ax[m].spines['right'].set_color('none') 
    ax[m].spines['top'].set_color('none') 
    m=m+1

    ax[m].plot(x,y_predicted[ts,1],'r',label='predicted') 
    ax[m].plot(x,y_true[ts,1],'k',label='true') 
    ax[m].tick_params(direction='in') 
    ax[m].legend()
    ax[m].set(ylabel='vertical eye {}'.format(output_type))
    ax[m].spines['right'].set_color('none') 
    ax[m].spines['top'].set_color('none') 

    fig.supxlabel('time (s)',y=0.07, fontsize=14)

### Format the neural data for the various types of models
The variable *X_flat* is used for feed-forward networks and *X* is used for recurrent neural networks

In [ ]:
num_timebins  =  neural_data.shape[0]
num_neurons   =  neural_data.shape[1]

bins_before = 6 #How many bins of neural data prior to the output are used for decoding
bins_current = 1 #Whether to use concurrent time bin of neural data
bins_after = 6 #How many bins of neural data after the output are used for decoding

# Format for recurrent neural networks (SimpleRNN, GRU, LSTM)
# Function to get the covariate matrix that includes spike history from previous bins
X = get_spikes_with_history(neural_data,bins_before,bins_after,bins_current)

# Format for Wiener Filter, Wiener Cascade, XGBoost, and Dense Neural Network
#Put in "flat" format, so each "neuron / time" is a single feature
X_flat=X.reshape(X.shape[0],(X.shape[1]*X.shape[2]))

### Choose what property of eye movement behavior you want to decode
Options include eye position, velocity, or acceleration (outputs have 2 columns for x and y components)

In [ ]:
# Position 
y = eye_pos

# Velocity
#y = eye_vel

# Acceleration
#y = eye_acc

print(y.shape)

### Split the data up into training, validation, and testing sets
Splitting data into train, validation, and test sets is an important practice in machine learning and data analysis because it helps to ensure that the model is not overfitting to the data and provides an unbiased estimate of the model's performance on new, unseen data 

In [ ]:
training_range=[0, 0.7]     # train = 70%
testing_range=[0.7, 0.85]   # test = 15%
valid_range=[0.85,1]        # validation = 15%

num_examples=X.shape[0]

training_set=np.arange(int(np.round(training_range[0]*num_examples))+bins_before,int(np.round(training_range[1]*num_examples))-bins_after)
testing_set=np.arange(int(np.round(testing_range[0]*num_examples))+bins_before,int(np.round(testing_range[1]*num_examples))-bins_after)
valid_set=np.arange(int(np.round(valid_range[0]*num_examples))+bins_before,int(np.round(valid_range[1]*num_examples))-bins_after)

#Get training data
X_train=X[training_set,:,:]
X_flat_train=X_flat[training_set,:]
y_train=y[training_set,:]

#Get testing data
X_test=X[testing_set,:,:]
X_flat_test=X_flat[testing_set,:]
y_test=y[testing_set,:]

#Get validation data
X_valid=X[valid_set,:,:]
X_flat_valid=X_flat[valid_set,:]
y_valid=y[valid_set,:]


# Certain models prefer the data be normalized or z-scored in a particular way
X_train_mean=np.nanmean(X_train,axis=0)
X_train_std=np.nanstd(X_train,axis=0)
X_train=(X_train-X_train_mean)/X_train_std
X_test=(X_test-X_train_mean)/X_train_std
X_valid=(X_valid-X_train_mean)/X_train_std

#Z-score "X_flat" inputs. 
X_flat_train_mean=np.nanmean(X_flat_train,axis=0)
X_flat_train_std=np.nanstd(X_flat_train,axis=0)
X_flat_train=(X_flat_train-X_flat_train_mean)/X_flat_train_std
X_flat_test=(X_flat_test-X_flat_train_mean)/X_flat_train_std
X_flat_valid=(X_flat_valid-X_flat_train_mean)/X_flat_train_std

#Zero-center outputs
y_train_mean=np.mean(y_train,axis=0)
y_train=y_train-y_train_mean
y_test=y_test-y_train_mean
y_valid=y_valid-y_train_mean

## Wiener filter

Let's start with the absolute simplest model, that doesn't require any parameterization.

The **Wiener filter** can be used to decode neural activity from the brain into an estimate of the stimulus or behavior that gave rise to that activity. The filter is trained on a set of labeled data, in which the neural activity and corresponding stimulus or behavior are known. Once the filter is trained, it can be applied to new, unlabeled data to decode the stimulus or behavior from the neural activity.
The Wiener filter is a simple but effective model that can be used to decode neural activity from a variety of brain regions, but doesn't perform *that* well given its strong assumptiosn of linearity and stationarity between inputs and outputs. 

In [ ]:
#Declare model
model_wf=WienerFilterDecoder()

#Fit model
model_wf.fit(X_flat_train,y_train)

#Get predictions
y_valid_predicted_wf=model_wf.predict(X_flat_valid)

plot_eyeTraces(0,1000,y_valid,y_valid_predicted_wf,'position')

### How do we quantity how well the decoding model performed?
To determine the goodness of fit of our models, we can use a version of $R^2$ that quantifies the fraction of variance accounted for in the predicted eye traces. The equation for $R^2$, where $\hat{y}_i$ and $y_i$ are the predicted and true values, respectively, is shown here.
$R^2 = 1 - \frac{\sum_{i}\left(\hat{y}_i - y_i\right)^2}{\sum_{i}\left(y_i - \bar{y}\right)^2}$

Specifically, $R^2$ represents the proportion of the variance in the dependent variable (e.g., the behavior or stimulus) that is explained by the independent variable (e.g., the neural activity) in the model. A higher $R^2$ value indicates a better fit between the model and the data, meaning that the model is better at explaining the variance in the dependent variable. However, it is important to note that a high $R^2$ value does not necessarily mean that the model is accurate or that it will generalize well to new data.

In [ ]:
#Get metric of fit
R2s_wf=get_R2(y_valid,y_valid_predicted_wf)
print('R2s:', R2s_wf)

Just from looking at the plot above, the Wiener filter does a pretty good job at predicting horizontal and vertical eye positions! And the $R^2$ value tells us how well the Wiener filter fits the *validation* data after being trained on the *training* dataset. The reason we test the model on data it has never seen before, like a validation or test set, is to properly assess the generalization performance of a model. By evaluating the model's performance on new data, it is possible to get a more accurate estimate of how well the model will perform on new, unseen data.

### So what about decoding models that require parameterization?
**Bayesian optimization** is a method for finding the optimal set of hyperparameters for decoding models, which can improve their performance by optimizing their ability to accurately decode neural activity. Hyperparameters are the parameters of the model that are set prior to training and are not learned from the data, such as the regularization strength or the number of neurons in the model.

Bayesian optimization works by constructing a probabilistic model of the objective function (e.g., the decoding accuracy) and using this model to select the next set of hyperparameters to evaluate. The method uses a combination of exploration (trying new sets of hyperparameters) and exploitation (using information gained from previous evaluations) to efficiently search the space of hyperparameters and find the optimal set. Using Bayesian optimization to parameterize decoding models can improve their performance by finding the optimal set of hyperparameters for the specific dataset and decoding problem at hand.

## Wiener cascade decoder
The **Wiener cascade decoder** is a variation of the Wiener filter model that allows for nonlinear input-output relationships, making it a more flexible and powerful decoder. It works by dividing the input signal (i.e., the neural activity) into a series of nonlinear subunits, each of which contributes to the output estimate. The output of each subunit is passed through a linear filter to account for the temporal dynamics of the neural response, and the outputs of all subunits are combined to produce the final estimate of the stimulus or behavior.

The advantage of the Wiener cascade decoder is that it can capture the nonlinear input-output relationships that are often present in neural activity, while still being based on a linear filtering framework. This allows it to capture more complex and subtle relationships between neural activity and behavior or stimuli, leading to improved decoding accuracy.

In [ ]:
# function for evaluating wiener cascade performance
def wc_evaluate(degree):
    model_wc=WienerCascadeDecoder(degree) #Define model
    model_wc.fit(X_flat_train,y_train) #Fit model
    y_valid_predicted_wc=model_wc.predict(X_flat_valid) #Validation set predictions
    return np.mean(get_R2(y_valid,y_valid_predicted_wc)) #R2 value of validation set (mean over x and y position/velocity)

In [ ]:
#Define Bayesian optimization, and set limits of hyperparameters 
# iter = iteration number, target = R^2 calculated in wc_evaluate, degree = tested parameter

BO_wc = BayesianOptimization(wc_evaluate, {'degree': (1, 20.99)}, verbose=1,allow_duplicate_points=True)
BO_wc.maximize(init_points=5, n_iter=5) # set number of initial runs (init_points) and subsequent tests (n_iter), and do the optimization

params = ((np.vstack((np.array([BO_wc.res[key]['target'] for key in range(len(BO_wc.res))]),np.array([(round(((BO_wc.res[key]['params']['degree'])*2))/2) for key in range(len(BO_wc.res))]))).T))
max_degree = int(params[np.argmax(params[:,0], axis=0),1])
print('degree = {}'.format(max_degree))

In [ ]:
# Wiener cascade model with the best fitted parameter from training set
model_wc=WienerCascadeDecoder(degree=max_degree)
model_wc.fit(X_flat_train,y_train)
y_test_predicted_wc=model_wc.predict(X_flat_test)

#Get metric of fit
R2s_wc=get_R2(y_test,y_test_predicted_wc)
print('R2s:', R2s_wc)

plot_eyeTraces(0,1000,y_test,y_test_predicted_wc,'position')

### So now that you've seen the basics of how the models work, now it's *your* turn!

Using a similar setup as above, plot the predicted and true eye traces for the next few models. 

For more help on implementing these decoding models, make sure you check out all of the example notebooks in the Kording Lab's GitHub repository (especially within the /Paper_code folder). 
https://github.com/KordingLab/Neural_Decoding

## 1a. Decode horizontal and vertical eye position using the XGBoost decoder.
* Describe how the model works and what assumptions it makes
* Define the evaluation function (*hint: these are in the Kording lab notebooks*)
* Run the Bayesian parameter optimization 
* Calculate the $R^2$ scoring metric 
* Plot the true and predicted eye traces

## 1b. Decode horizontal and vertical eye position using the SVR decoder.
* Describe how the model works and what assumptions it makes
* Define the evaluation function (*hint: these are in the Kording lab notebooks*)
* Run the Bayesian parameter optimization 
* Calculate the $R^2$ scoring metric 
* Plot the true and predicted eye traces

## 1c. Decode horizontal and vertical eye position using the DNN decoder.
* Describe how the model works and what assumptions it makes
* Define the evaluation function (*hint: these are in the Kording lab notebooks*)
* Run the Bayesian parameter optimization 
* Calculate the $R^2$ scoring metric 
* Plot the true and predicted eye traces

## 1d. Make a plot that compares the $R^2$ values from these 5 types of models side-by-side
* Plot: x-axis = model type, y-axis = $R^2$ value
* Which model was best?

## 2a. Run each of the models, but now decode out eye velocity instead of position
* Store the $R^2$ values for each model type
* Make a plot, just like in 1d, that compares the $R^2$ values from the 5 model types side-by-side

## 2b. Run each of the models, but now decode out eye acceleration instead of position
* Store the $R^2$ values for each model type
* Make a plot, just like in 1d, that compares the $R^2$ values from the 5 model types side-by-side

## Use cross-validation to assess model generalization performance
Up to this point, you've split the data into training/validation/testing sets once. But, for statistical power and generalizability we need to run each model multiple times on different subsets of the data.  

**Cross-validation** is important for decoding models because it provides an estimate of how well the model is likely to perform on new, unseen data. In cross-validation, the available data is split into multiple subsets or "folds," and the model is trained on some folds and tested on others. This allows for an assessment of the model's performance on data that was not used during training, which provides an estimate of the model's ability to generalize to new data.

In the context of decoding models, cross-validation can be used to assess how well the model is able to decode neural activity into a behavior or stimulus of interest. By using a portion of the available data as a test set, the model's decoding accuracy can be evaluated on new, independent data, which provides a more accurate estimate of the model's generalization performance.

*Hints on how to do this are in the Neural_Decoding/Paper_code/ManyDecoders_FullData.ipynb notebook on the Kording Lab GitHub*

In [ ]:
valid_range_all=[[0,.1],[.1,.2],[.2,.3],[.3,.4],[.4,.5],
                 [.5,.6],[.6,.7],[.7,.8],[.8,.9],[.9,1]]
testing_range_all=[[.1,.2],[.2,.3],[.3,.4],[.4,.5],[.5,.6],
                 [.6,.7],[.7,.8],[.8,.9],[.9,1],[0,.1]]
training_range_all=[[[.2,1]],[[0,.1],[.3,1]],[[0,.2],[.4,1]],[[0,.3],[.5,1]],[[0,.4],[.6,1]],
                   [[0,.5],[.7,1]],[[0,.6],[.8,1]],[[0,.7],[.9,1]],[[0,.8]],[[.1,.9]]]

num_folds=len(valid_range_all) # Number of cross validation folds

## 3a. Remake the plots from 1d, 2a, and 2b... but now with ten $R^2$ values for each model type 
* Each model will have 10 $R^2$ values, one for each cross-validation fold
* Plot the mean $R^2$ value for each model type, and include error bars 

In [ ]:
# Initialize empty lists to store results from each cross-validation fold
r2_wf=[]
#wc, xgb, etc...

for i in range(num_folds): #Loop through the folds
    print(i)
    # Split the data into train, valid, and test sets using the ranges above
    
    # Preprocess the data, as we did before (z-score X inputs, z-score X_flat inputs, zero-center outputs, z-score outputs for SVR)
    
    # Run each model type (train on training data, fit parameters with validation data, test with test data)
    # The evaluation functions are the exact same as before
    
    #def wc_MODELTYPE(PARAMS): 
        #model_XX=DECODERNAME(PARAMS) #Define model
        #model_XX.fit(X_flat_train,y_train) #Fit model
        #y_valid_predicted_XX=model_XX.predict(X_flat_valid) #Validation set predictions
        #return np.mean(get_R2(y_valid,y_valid_predicted_wc)) #R2 value of validation set (mean over x and y position/velocity)

    # Do bayesian optimization

    # Run model w/ above hyperparameters on test set
    
    # Calculate R^2 value and store

      